In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# Run only if GPU is found
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

Found GPU at: /device:GPU:0


In [0]:
#@title Default title text
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2
# Customization
l = 12
num_filter = 48

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 40s 0us/step


In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3, 3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 48)   192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 48)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# Load Weights
weights_file_name = "weights_best.hdf5"
# model.load_weights(weights_file_name)

In [0]:
# Determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# Add Checkpoint
checkpoint = ModelCheckpoint(weights_file_name, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 427s 9ms/step - loss: 1.4560 - acc: 0.4674 - val_loss: 1.8342 - val_acc: 0.4299

Epoch 00001: val_acc improved from -inf to 0.42990, saving model to weights_best.hdf5
Epoch 2/50
14464/50000 [=======>......................] - ETA: 4:29 - loss: 1.0583 - acc: 0.6191

50000/50000 [==============================] - 406s 8ms/step - loss: 0.9886 - acc: 0.6445 - val_loss: 1.0253 - val_acc: 0.6538

Epoch 00002: val_acc improved from 0.42990 to 0.65380, saving model to weights_best.hdf5
Epoch 3/50
30976/50000 [=================>............] - ETA: 2:24 - loss: 0.8230 - acc: 0.7056

50000/50000 [==============================] - 407s 8ms/step - loss: 0.8046 - acc: 0.7114 - val_loss: 1.3961 - val_acc: 0.5919

Epoch 00003: val_acc did not improve from 0.65380
Epoch 4/50
40576/50000 [=======================>......] - ETA: 1:10 - loss: 0.6943 - acc: 0.7539

50000/50000 [==============================] - 405s 8ms/step - loss: 0.6839 - acc: 0.7582 - val_loss: 1.4505 - val_acc: 0.6379

Epoch 00004: val_acc did not improve from 0.65380
Epoch 5/50
43648/50000 [=========================>....] - ETA: 47s - loss: 0.6090 - acc: 0.7846

50000/50000 [==============================] - 406s 8ms/step - loss: 0.6040 - acc: 0.7867 - val_loss: 0.9265 - val_acc: 0.7243

Epoch 00005: val_acc improved from 0.65380 to 0.72430, saving model to weights_best.hdf5
Epoch 6/50
38912/50000 [======================>.......] - ETA: 1:23 - loss: 0.5477 - acc: 0.8061

...

50000/50000 [==============================] - 406s 8ms/step - loss: 0.0785 - acc: 0.9720 - val_loss: 0.6868 - val_acc: 0.8657

Epoch 00046: val_acc did not improve from 0.87760
Epoch 47/50
44672/50000 [=========================>....] - ETA: 40s - loss: 0.0754 - acc: 0.9730

50000/50000 [==============================] - 406s 8ms/step - loss: 0.0751 - acc: 0.9733 - val_loss: 0.6235 - val_acc: 0.8720

Epoch 00047: val_acc did not improve from 0.87760
Epoch 48/50
44672/50000 [=========================>....] - ETA: 40s - loss: 0.0705 - acc: 0.9744

50000/50000 [==============================] - 406s 8ms/step - loss: 0.0708 - acc: 0.9743 - val_loss: 0.7801 - val_acc: 0.8595

Epoch 00048: val_acc did not improve from 0.87760
Epoch 49/50
44672/50000 [=========================>....] - ETA: 40s - loss: 0.0679 - acc: 0.9757

50000/50000 [==============================] - 406s 8ms/step - loss: 0.0689 - acc: 0.9754 - val_loss: 0.7460 - val_acc: 0.8589

Epoch 00049: val_acc did not improve from 0.87760
Epoch 50/50
44672/50000 [=========================>....] - ETA: 40s - loss: 0.0652 - acc: 0.9775

50000/50000 [==============================] - 406s 8ms/step - loss: 0.0676 - acc: 0.9765 - val_loss: 0.8107 - val_acc: 0.8456

Epoch 00050: val_acc did not improve from 0.87760


In [0]:
# Ensure test runs on best model
from keras.models import load_model
best_model = load_model(weights_file_name)

In [16]:
# Test the model
score = best_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 38s 4ms/step
Test loss: 0.6446526644617319
Test accuracy: 0.8776


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')